# Polyglot. Find names from Name_entity_person

In [2]:
#=====Download this before polyglot usage=====
from polyglot.downloader import downloader
downloader.supported_tasks(lang="de")
downloader.download("embeddings2.de")
downloader.download("pos2.de")
downloader.download("ner2.de")
#=========================

[polyglot_data] Downloading package ner2.de to C:\Anaconda\Lib\site-
[polyglot_data]     packages\polyglot\data...


True

In [100]:
import polyglot
from polyglot.text import Text, Word
import json

def identify_name(raw_text):
    list_of_personal_names=[]
    text = Text(raw_text)
    text._BaseBlob__lang=polyglot.detect.base.Language.from_code("de") #we assign to the text German language!!
    try:
        if text.language.code!="de":
            print text.language.code
    except Exception, e:
        print 'failed in language detection'
        print str(e)
        return list_of_personal_names,True
    try:
        for t in text.entities:
            if (t.tag=="I-PER"):
                if (len(t[0])!=1)&(t[0].istitle()):
                    list_of_personal_names.append(t[0] if len(t)==1 else " ".join(i for i in t))# if i.isalpha()))   
        return list_of_personal_names,False
    except Exception, e:
        print 'failed in the entities'
        print str(e)
        return list_of_personal_names,True



In [104]:
#====progressbar   
#from progress.bar import Bar
#bar = Bar('Processing', max=20)
#bar.start()
#============


with open('de/wiki/all_data_from pages.json', 'r') as in_f:
    Dump=json.load(in_f)
names={}
error_list=[]
for i in Dump:
    #bar.next()
    ne,error=identify_name(Dump[i]["text"].encode("utf-8").decode("utf-8").replace("\n",", "))
    if error:
        print "error at text of:",i
        error_list.append(i)
    else:
        name_list=list(set(ne))#delete duplicates
        name_list=filter(lambda k: i not in k, name_list)#clean from name of article
        name_list=[p for p in name_list if not((" e ." in p)|(" e." in p))]#clean from e.V.
        name_list=[p for p in name_list if not(p in ["Erw .","Hon .","Sta .","Ed .","Ma ?","Ass .","Kyu -","Éd .".decode("utf-8"),"Aja -"])] 
        
        #replace non aplchabetical values at the end of full name
        new_p=[]
        old_p=[]
        for p in name_list:
            if not(p[-1:].isalpha()):
                new_p_=p[:-2]
                if new_p_[-1:]==".":
                    new_p_=p[:-4]
                #print p, new_p
                new_p.append(new_p_)
                old_p.append(p)
        name_list=list(set(name_list)-set(old_p))
        name_list.extend(new_p)
        name_list=list(set(name_list)) 
        name_list=[p for p in name_list if not(("Ed"==p)|("Verl"== p))]#clean Ed and Verl
        name_list=[p for p in name_list if not(("Mr"==p)|("Co"==p)|("Ch"==p)|("Chr"==p)|("Sc"==p)|("St"==p))]#clean Mr Co Ch Chr Sc St
        names[i]=name_list
#bar.finish()

failed in the entities
'ascii' codec can't encode character u'\xd6' in position 0: ordinal not in range(128)
error at text of: Flugbegleiter


In [105]:
#get from error data# scip not readable char
#error_list[0]#"Flugbegleiter"
ne,error=identify_name(Dump["Flugbegleiter"]["text"][:14036].replace("\n",", ")+" "+Dump["Flugbegleiter"]["text"][14050:].replace("\n",", "))
if error:
    print "error at text of:",i
    error_list.append(i)
else:
    ne=list(set(ne))
    ne=filter(lambda k: "Flugbegleiter" not in k, ne)#clean from name of article
    ne=[p.replace(" .","") for p in ne]#clean from " ."
    print ne
    names["Flugbegleiter"]=ne

[u'Maria Weckesser', u'Saftschubse', u'Strahlenexposition', u'Passagiersitzplatz', u'Nelly Diener', u'Heather Poole', u'Guinness', u'Flugg\xe4sten', u'Kathrin Leineweber', u'Ellen Church', u'Ingo Matuschek', u'Annette Lies', u'Ron Akana', u'Andrea Brandl', u'Annette', u'Gray', u'Piper']


In [106]:
print len(names)#,len(Dump)
with open('de/wiki/all_name_entity_person_polyglot.json', 'w') as out:
    json.dump(names, out, indent=4, sort_keys=True,encoding="utf-8")

885


## filter first names and lastnames when repeats

In [146]:
#if first name of full name
#if last name of full name
#if full name ends or beginsw ith this name
###in more detail:
###first two names, check that there are two names and check at the begining and end of full name
###Ex: Matthäus Merian => Matthäus Merian der Ältere

#if second/third name in full name

#add Marcel Marceau, del all with Marceau


import json


with open('de/wiki/all_name_entity_person_polyglot.json', 'r') as in_f:
    names=json.load(in_f,encoding="utf-8")


from collections import defaultdict
to_del=defaultdict(list)
to_add=defaultdict(list)
for prof in names:
    #print prof
    for p in names[prof]:
        for p2 in names[prof]:
            if (p not in to_del[prof]):#|(p2 not in to_del[prof]):
                if p!=p2:
                    if p in p2:
                        if p+"s"==p2:
                            to_del[prof].append(p2) 
                            #print p,"=>",p2
                        elif p+"’s".decode("utf-8")==p2:
                            to_del[prof].append(p2)
                            #print p,"=>",p2  
                        elif (p2.split(" ")[0]==p):
                            to_del[prof].append(p) #exclude p from list
                            #print p,"=>",p2
                        elif (p2.split(" ")[-1]==p):
                            to_del[prof].append(p)
                            #print p,"=>",p2
                        elif (len(p.split(" "))==2)&(p2[:len(p)]==p):
                            #print p,"=>",p2
                            #print prof
                            to_del[prof].append(p)
                        elif (p2[-len(p):]==p):
                            #print p,"=>",p2
                            to_del[prof].append(p)
                        elif (p2[-len(p)-1:]==p+"s"):
                            #print p,"=>",p2
                            to_del[prof].append(p2)
                            to_del[prof].append(p)
                            to_add[prof].append(p2[:-1])
                        elif ("Marceau" in p)& ("Marcel Marceau" in p2):
                            to_del[prof].append(p2)
                            to_del[prof].append(p)
                            to_add[prof].append("Marcel Marceau")
                        elif (p in p2.split(" "))&(len(p2.split(" "))>1):
                            #print p,"=>",p2
                            to_del[prof].append(p)
                        #else:
                            #print p,"=>",p2
    to_del[prof]=list(set(to_del[prof]))
    to_add[prof]=list(set(to_add[prof]))
#del
for prof in to_del:
    if len(to_del[prof])>0:
        #print to_del[prof]
        names[prof]=[n for n in names[prof] if n not in to_del[prof]]
        names[prof]=list(set(names[prof]))


print"===Add"
#add
for prof in to_add:
    if len(to_add[prof])>0:
        #print to_add[prof]
        names[prof].extend(to_add[prof])
        names[prof]=list(set(names[prof]))

===Add


In [147]:
import re
from collections import defaultdict

#parse 

def del_and_add_to_dict(names,names_add,names_del):
    print "We add:",len(names_add),"new instances and del ",len(names_del)#1,9
    for prof in names_del:
        if len(names_del[prof])>0:
            names[prof]=[n for n in names[prof] if n not in names_del[prof]]
            names[prof]=list(set(names[prof]))
    for prof in names_add:
        if len(names_add[prof])>0:
            names[prof].extend(names_add[prof])
            names[prof]=list(set(names[prof])) 
    return names



names_add=defaultdict(list)
names_del=defaultdict(list)
for prof in names:
    for p in names[prof]:
        if "Verl" in p:
            if " . Laaber Verlag" in p:
                new_p=p.replace(" . Laaber Verlag","")
                if new_p not in names[prof]:
                    names_add[prof].append(new_p)
            names_del[prof].append(p)
        elif " . " in p:
            new_p=p.replace(" . ", ". ")
            names_del[prof].append(p)
            if new_p not in names[prof]:
                names_add[prof].append(new_p)
        elif (". " in p):
            pass
        elif ("." in p) | ("&" in p) |("/" in p) | (")" in p):
            names_del[prof].append(p)
            print p
        elif "–".decode("utf-8") in p:#its not a minus  it is deffise
            new_p=p.split("–".decode("utf-8"))[0].strip()
            if (new_p not in names[prof]) & (new_p!="Attrice"):
                names_add[prof].append(new_p)
            names_del[prof].append(p)
        elif (p=="Sta")|(p=="Erw")|(p=="Ed")|(p=="Ma")|(p=="Ass")|(("Law"in p)&("urist"in prof))\
        |((p=="Jun")&(prof=="Arzt"))|((p=="Kyu")&(prof=="Polizeivollzugsbeamter"))|((p=="May")&(prof=="Butler"))\
        |((p=="Arx")&(prof=="Diakon"))|((p=="Wes")&(prof=="Unternehmensberater"))|(p=="Éd".decode("utf-8"))\
        |((p=="Aja")&(prof=="Richter"))|((p=="Due")&(prof=="Wirtschaftsprüfer".decode("utf-8")))\
        |((p=="Hon")&(prof=="Professor")):#Law/Laws in Juist
            names_del[prof].append(p)
            #print p,",",new_p,",",prof
            
print len(names_add),len(names_del)#79,113
names=del_and_add_to_dict(names,names_add,names_del) 

names_add=defaultdict(list)
names_del=defaultdict(list)
for prof in names:
    for p in names[prof]:
        if "verlag" in p:
            if ". Henschelverlag" in p:
                new_p=p.replace(". Henschelverlag","").strip()
                if new_p not in names[prof]:
                    names_add[prof].append(new_p)
            names_del[prof].append(p)
            
names=del_and_add_to_dict(names,names_add,names_del) 

Emmerich )
0 1
===Add


In [148]:
print len(names)#,len(Dump)
with open('de/wiki/all_name_entity_person_polyglot.json', 'w') as out:
    json.dump(names, out, indent=4, sort_keys=True,encoding="utf-8")

885


# Identify gender by first names

In [1]:
import pandas as pd
import math

names_2=pd.read_csv("gender_names_40000.txt",sep=" ",header=None,index_col=False,names=range(1,84),encoding="utf-8")
names_2=names_2[names_2.columns[:6]]
def move_value(x):
    if x[1] in ["1M","1F","?M","?F"]:
        return x[2]
    else:
        return x[3]
names_2["first_name"]=names_2.apply(lambda x:  move_value(x),axis=1) #all M F in third column
names_3=names_2[~names_2[1].isin(["?","?F","?M","="])]
names_3 = names_3[[1,'first_name']]
names_4 = names_3[[1,'first_name']]
names_4["first_name"]=names_4.apply(lambda x: x.first_name.replace("<r^>",u'\u0159').replace(u'\u2030',u'\xe4').replace("<l/>",u'\u0142').title().replace("è".decode("utf-8"),"é".decode("utf-8")).\
                                    replace(u'\xec',u'\xed'),axis=1)
#Examples wher names_4 is usefull:
#u'Napoléon',u'Karolína',u'Bedřich',u'Czesław',u'Bartholomäus',u'Mathäus',u'Kätchen',u'Bolesław',u'Stanisław'

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (4,5,11,12,13,14,26,27,28,29,30,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import pandas as pd
import numpy as np
import json


with open('de/wiki/all_name_entity_person_polyglot.json', 'r') as in_f:
    names=json.load(in_f,encoding="utf-8")

df_names_gender=pd.read_csv("name_gender_genderiz.txt", sep=" ",header=None,
                            names=["name","gender","conf","n"],index_col=False,encoding="utf-8")
df_names_gender.gender=df_names_gender.gender.str.replace("\[\(u'","").str.replace("\',","") 
df_names_gender.conf=df_names_gender.conf.str.replace("u'","").str.replace("\',","").convert_objects(convert_numeric=True)

df_names_gender_None=df_names_gender[df_names_gender.gender.str.contains("None")]
df_names_gender=df_names_gender[df_names_gender.gender.str.contains("male")]

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [5]:
print len(df_names_gender)#35320


35320


,name,gender,conf,n
292,Klaus-Peter,male,0,0.0)]
317,Hans-Michael,male,0,0.0)]
717,Abdulwahed,male,0,0.0)]
778,(Jeff),male,0,0.0)]
871,Schahram,male,0,0.0)]
892,Aryya,male,0,0.0)]
910,Hanns-Christian,male,0,0.0)]
943,Maria-Eugenia,female,0,0.0)]
3410,Khosraviyan,male,0,0.0)]
3411,Davari,male,0,0.0)]


In [433]:
#We have such examples:
#"K\u00f6nig Alexander I","Deutschland Egon Erwin Kisch"
#We look into all words in a name and store first occurency in DB


from collections import defaultdict

names_gender=defaultdict(dict)
all_other=[]
all_other_1=[]
all_other_2=[]
all_other_none=[]
for prof in names:
    for p in names[prof]:
        if (p.split(" ",1)[0] in df_names_gender.name.values):
            gender=df_names_gender[df_names_gender.name==p.split(" ",1)[0]].gender.values[0]
            if gender not in names_gender[prof]:
                names_gender[prof][gender]=[]
            names_gender[prof][gender].append(p)   
        elif p.split(" ",1)[0] in names_3.first_name.values: 
            g=names_3[names_3.first_name==p.split(" ",1)[0]][1].values[0]
            if (g=="F")|(g=="1F"):
                gender="female"
            else:
                gender="male"
            if gender not in names_gender[prof]:
                names_gender[prof][gender]=[] 
            names_gender[prof][gender].append(p)
        elif p.split()[0] in names_4.first_name.values: 
            g=names_4[names_4.first_name==p.split(" ",1)[0]][1].values[0]
            if (g=="F")|(g=="1F"):
                gender="female"
            else:
                gender="male"
            if gender not in names_gender[prof]:
                names_gender[prof][gender]=[] 
            names_gender[prof][gender].append(p)
        elif ((p.split(" ",1)[0][-1:]=="s") & (p.split(" ",1)[0][:-1] in df_names_gender.name.values)&\
             (p not in ["Criss Angel","Mimes","Tages","Christians","Dantes","Hors","Reuters"])):
            gender=df_names_gender[df_names_gender.name==p.split(" ",1)[0][:-1]].gender.values[0]
            if len(p.split())==1:
                real_name=p.split(" ",1)[0][:-1]
            else:
                first_name,last_name=p.split(" ",1)
                real_name=first_name[:-1]+" "+last_name
            #print "=",p,",",real_name,",",prof
            #check weather real_name in the list of names
            if [i for i in names[prof] if ((p.split()[0][:-1] in i)&(i!=p)) ]:
                pass
            else:
                if gender not in names_gender[prof]:
                    names_gender[prof][gender]=[]
                names_gender[prof][gender].append(real_name)
        elif (len(p.split())<3)&((p.split()[0] in df_names_gender_None.name.values)):
            all_other_none.append(p+" , "+prof)
            pass
        elif (len(p.split(" "))>1):
            if (len(p.split())==2) & (not ( (p.split()[1].istitle())&(len(p.split()[1])>1))):
                all_other.append(p+" , "+prof)
                #print p
            #if the second word starts from capital letter and more than one char
            else:
                found=""
                for i in p.split()[1:]:
                    if (i.istitle())& (len(i)>1):
                        if i in df_names_gender.name.values:
                            #print "!!found! in df_names_gender:",p
                            gender=df_names_gender[df_names_gender.name==i].gender.values[0]
                            if gender not in names_gender[prof]:
                                names_gender[prof][gender]=[]
                            names_gender[prof][gender].append(p)
                            found=p
                            break
                        elif i in names_3.first_name.values:
                            #print "!!found! in names_3:",p
                            g=names_3[names_3.first_name==i][1].values[0]
                            if (g=="F")|(g=="1F"):
                                gender="female"
                            else:
                                gender="male"
                            if gender not in names_gender[prof]:
                                names_gender[prof][gender]=[] 
                            names_gender[prof][gender].append(p)
                            found=p
                            break
                if len(found)==0:
                    all_other_2.append(p+" , "+prof)
                    #print p+" , "+prof
                        
        else:
            all_other_1.append(p+" , "+prof)
#print all_other,all_other_1,all_other_2,all_other_none

In [442]:
#store everything filtered from algorithm
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('de/all_not_person_polyglot.pckl', 'w') as f:
    pickle.dump([all_other,all_other_1,all_other_2,all_other_none], f)

# Getting back the objects:
with open('de/all_not_person_polyglot.pckl') as f:
    all_other,all_other_1,all_other_2,all_other_none = pickle.load(f)

In [443]:
#add counts
for prof in names_gender:
    if names_gender[prof].has_key("male"):
        names_gender[prof]["male_amount"]=len(names_gender[prof]["male"])
    if names_gender[prof].has_key("female"):
        names_gender[prof]["female_amount"]=len(names_gender[prof]["female"])
        
        
with open('de/all_person_polyglot_new.json', 'w') as out:
    json.dump(names_gender, out, indent=4, sort_keys=True)

# Stanford approach==> Does not work

In [6]:
import nltk
from nltk.tag.stanford import NERTagger
nltk.internals.config_java("C:\\Java\\jdk1.8.0_25\\bin\\java.exe")#jre1.8.0_66
path_to_model ="C:\Users\Lelka\Downloads\stanford-ner-2015-12-09\stanford-ner-2015-12-09\classifiers\english.all.3class.distsim.crf.ser.gz"
path_to_jar = "C:\Users\Lelka\Downloads\stanford-ner-2015-12-09\stanford-ner-2015-12-09\stanford-ner-3.6.0.jar"
st = NERTagger(path_to_model,path_to_jar)#, 'stanford-ner/stanford-ner.jar')
#st = NERTagger("C:\Users\Lelka\Downloads\stanford-german-2015-10-14-models.jar",
#               "C:\Users\Lelka\Downloads\stanford-ner-2015-12-09\stanford-ner-2015-12-09\stanford-ner.jar")
#download from http://nlp.stanford.edu/software/CRF-NER.html

#st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz')

#from nltk.tag import StanfordPOSTagger
#st = StanfordPOSTagger('english-bidirectional-distsim.tagger') 
text = """YOUR TEXT GOES HERE"""

for sent in nltk.sent_tokenize(text):
    tokens = nltk.tokenize.word_tokenize(sent)
    tags = st.tag(tokens)
    for tag in tags:
        if tag[1]=='PERSON': print tag

[Found C:\Java\jdk1.8.0_25\bin\java.exe: C:\Java\jdk1.8.0_25\bin\java.exe]
CRFClassifier invoked on Thu Mar 03 01:54:17 CET 2016 with arguments:
   -loadClassifier C:\Users\Lelka\Downloads\stanford-ner-2015-12-09\stanford-ner-2015-12-09\classifiers\english.all.3class.distsim.crf.ser.gz -textFile c:\users\lelka\appdata\local\temp\tmptwpxy4 -outputFormat slashTags
loadClassifier=C:\Users\Lelka\Downloads\stanford-ner-2015-12-09\stanford-ner-2015-12-09\classifiers\english.all.3class.distsim.crf.ser.gz
textFile=c:\users\lelka\appdata\local\temp\tmptwpxy4
outputFormat=slashTags
Exception in thread "main" java.lang.NoClassDefFoundError: org/slf4j/LoggerFactory
	at edu.stanford.nlp.io.IOUtils.<clinit>(IOUtils.java:42)
	at edu.stanford.nlp.ie.AbstractSequenceClassifier.loadClassifier(AbstractSequenceClassifier.java:1484)
	at edu.stanford.nlp.ie.AbstractSequenceClassifier.loadClassifierNoExceptions(AbstractSequenceClassifier.java:1497)
	at edu.stanford.nlp.ie.crf.CRFClassifier.main(CRFClassifier

OSError: Java command failed!

In [6]:
from nltk.parse.stanford import StanfordParser
from nltk.parse.stanford import StanfordDependencyParser
from nltk.parse.stanford import StanfordNeuralDependencyParser
from nltk.tag.stanford import StanfordPOSTagger, StanfordNERTagger
from nltk.tokenize.stanford import StanfordTokenizer

In [79]:
from nltk.tag.stanford import StanfordPOSTagger
st = StanfordPOSTagger('english-bidirectional-distsim.tagger')

LookupError: 

===========================================================================
  NLTK was unable to find stanford-postagger.jar! Set the CLASSPATH
  environment variable.

  For more information, on stanford-postagger.jar, see:
    <http://nlp.stanford.edu/software>
===========================================================================

In [2]:
import nltk
from nltk.tag import StanfordNERTagger
#nltk.internals.config_java("C:\\Java\\jdk1.8.0_73\\bin\\java.exe")
st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz',
                       path_to_jar="c:\stanford\stanford-ner-2015-12-09\stanford-ner-3.6.0.jar") 
st.tag('Rami Eid is studying at Stony Brook University in NY'.split())

CRFClassifier invoked on Thu Mar 03 14:40:03 CET 2016 with arguments:
   -loadClassifier C:\Stanford\stanford-ner-2015-12-09\classifiers\english.all.3class.distsim.crf.ser.gz -textFile c:\users\zagovooa\appdata\local\temp\tmpmepzqe -outputFormat slashTags -tokenizerFactory edu.stanford.nlp.process.WhitespaceTokenizer -tokenizerOptions "tokenizeNLs=false" -encoding utf8
tokenizerFactory=edu.stanford.nlp.process.WhitespaceTokenizer
tokenizerOptions="tokenizeNLs=false"
loadClassifier=C:\Stanford\stanford-ner-2015-12-09\classifiers\english.all.3class.distsim.crf.ser.gz
encoding=utf8
textFile=c:\users\zagovooa\appdata\local\temp\tmpmepzqe
outputFormat=slashTags
Exception in thread "main" java.lang.NoClassDefFoundError: org/slf4j/LoggerFactory
	at edu.stanford.nlp.io.IOUtils.<clinit>(IOUtils.java:42)
	at edu.stanford.nlp.ie.AbstractSequenceClassifier.loadClassifier(AbstractSequenceClassifier.java:1484)
	at edu.stanford.nlp.ie.AbstractSequenceClassifier.loadClassifierNoExceptions(AbstractSequ

OSError: Java command failed : ['C:\\Java\\jdk1.8.0_73\\bin\\java.exe', '-mx1000m', '-cp', 'c:\\stanford\\stanford-ner-2015-12-09\\stanford-ner-3.6.0.jar', 'edu.stanford.nlp.ie.crf.CRFClassifier', '-loadClassifier', 'C:\\Stanford\\stanford-ner-2015-12-09\\classifiers\\english.all.3class.distsim.crf.ser.gz', '-textFile', 'c:\\users\\zagovooa\\appdata\\local\\temp\\tmpmepzqe', '-outputFormat', 'slashTags', '-tokenizerFactory', 'edu.stanford.nlp.process.WhitespaceTokenizer', '-tokenizerOptions', '"tokenizeNLs=false"', '-encoding', 'utf8']

# NLTK Approach (too slow)

In [1]:
import nltk
#nltk.download()
print(nltk.__version__)

3.1


In [21]:
import nltk
from nltk.tree import Tree
import re

def identify_person_in_text(text):
    person_list = []
    for sentence in nltk.sent_tokenize(text):
        #print sentence
        sent1 = nltk.word_tokenize(sentence)
        sent2 = nltk.pos_tag(sent1)
        sent3 =  nltk.ne_chunk(sent2,binary = False)#, binary=True)
        #print sent3
        #sent3.draw()
        person_list.append(re.findall(r'PERSON\s(.*?)/',str(sent3)))
    return person_list
def identify_person_in_text2(text):
    person_list = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))).subtrees(filter=lambda t: t.label() == 'PERSON'):
            person_list.append(' '.join(c[0] for c in chunk.leaves()))
    return person_list

def identify_person_in_text3(text):
    person_list = []
    for sentence in nltk.sent_tokenize(text):
        #print sentence
        sent1 = nltk.word_tokenize(sentence)
        sent2 = nltk.pos_tag(sent1)
        sent3 =  nltk.ne_chunk(sent2,binary = False)#, binary=True)
        person = []
        name = ""
        for subtree in sent3.subtrees(filter=lambda t: t.label() == 'PERSON'):
            for leaf in subtree.leaves():
                person.append(leaf[0])
                #print person
            if len(person) > 1: #avoid grabbing lone surnames
                for part in person:
                    name += part + ' '
                if name[:-1] not in person_list:
                    person_list.append(name[:-1])
                name = ''
            else:
                person_list.append(person[0])
            person = []
    return person_list


#######==========Example 1

import progressbar
from itertools import groupby
import json

bar = progressbar.ProgressBar(maxval=5, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

text=u"In Großbritannien war Gandhi und Arnim mit dem westlichen Lebensstil vertraut geworden. Ralf Lämmel war nett. Olga Zagovora kommt auch."
person_nltk={}

with open('de/wiki/all_data_from pages.json', 'r') as in_f:
    Dump=json.load(in_f)
text=Dump["Abfallbeauftragter"]["text"]
j=0
bar.start()
if True:#for article in Dump:
    bar.update(j+1)
    if j<=3:
        #text=Dump[article]["text"]
        person_nltk[article]=identify_person_in_text2(text)
    #else:
        #break
        
    j+=1
bar.finish()

In [23]:
#============ Example 2
import timeit
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

text1="I love you. Irma loves Ralf."
text2=u"In Großbritannien war Gandhi und Arnim mit dem westlichen Lebensstil vertraut geworden. Alfred Lämmel war nett. Olga Zagovora kommt auch."

wrapped1=wrapper(identify_person_in_text,text1)
print timeit.timeit(wrapped1, number=3)
wrapped2=wrapper(identify_person_in_text2,text1)
print timeit.timeit(wrapped2, number=3)
wrapped3=wrapper(identify_person_in_text3,text1)
print timeit.timeit(wrapped3, number=3)
#wrapped4=wrapper(identify_person_in_text3,text2)
#print timeit.timeit(wrapped4,number=3)

61.6410712819
65.3662776135
65.4470353946
96.695457625
